In \[1\]:

    import numpy as np

    a = [0, 1, 2, 5, 6, 8, 9]
    b = [0, 2, 3, 4, 5, 7, 9]

In \[2\]:

    def jaccard(list1, list2):
        intersection = len(list(set(list1).intersection(list2)))
        union = (len(list1) + len(list2)) - intersection
        return float(intersection) / union

    jaccard(a, b)

Out\[2\]:

    0.4

In \[5\]:

    import numpy

    def levenshteinDistanceDP(token1, token2):
        distances = numpy.zeros((len(token1) + 1, len(token2) + 1))

        for t1 in range(len(token1) + 1):
            distances[t1][0] = t1

        for t2 in range(len(token2) + 1):
            distances[0][t2] = t2
            
        printDistances(distances, len(token1), len(token2))
        return 0

    def printDistances(distances, token1Length, token2Length):
        for t1 in range(token1Length + 1):
            for t2 in range(token2Length + 1):
                print(int(distances[t1][t2]), end=" ")
            print()
            
    levenshteinDistanceDP("kelm", "hello")

    0 1 2 3 4 5 
    1 0 0 0 0 0 
    2 0 0 0 0 0 
    3 0 0 0 0 0 
    4 0 0 0 0 0 

Out\[5\]:

    0

In \[6\]:

    doc_J = "Julie loves me more than Linda loves me"

    doc_L = "Jane likes me more than Julie loves me"


    documents = [doc_J, doc_L]

In \[9\]:

    from sklearn.feature_extraction.text import CountVectorizer
    import pandas as pd

    count_vectorizer = CountVectorizer(stop_words='english')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(documents)


    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=['doc_J', 'doc_L'])
    df

Out\[9\]:

|        | jane | julie | likes | linda | loves | me  | more | than |
|--------|------|-------|-------|-------|-------|-----|------|------|
| doc\_J | 0    | 1     | 0     | 1     | 2     | 2   | 1    | 1    |
| doc\_L | 1    | 1     | 1     | 0     | 1     | 2   | 1    | 1    |

In \[10\]:

    from sklearn.metrics.pairwise import cosine_similarity
    print(cosine_similarity(df, df))

    [[1.         0.82158384]
     [0.82158384 1.        ]]

In \[11\]:

    from sklearn.feature_extraction.text import TfidfVectorizer
    corpus = [
        'This is the first document.',
        'This document is the second document.',
        'And this is the third one.',
        'Is this the first document?', ]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    print(vectorizer.get_feature_names())
    print(X.shape)

    ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
    (4, 9)

In \[12\]:

    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    %matplotlib inline
    import seaborn as sns
    import re

    import pickle 

    import time


    from nltk.tokenize import TweetTokenizer # doesn't split at apostrophes
    import nltk
    from nltk import Text
    from nltk.tokenize import regexp_tokenize
    from nltk.tokenize import word_tokenize  
    from nltk.tokenize import sent_tokenize 
    from nltk.corpus import stopwords
    from nltk.stem import WordNetLemmatizer
    from nltk.stem import PorterStemmer


    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.linear_model import LogisticRegression 
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.multiclass import OneVsRestClassifier


    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import train_test_split
    from sklearn import metrics
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import make_pipeline

In \[17\]:

    txt = ["He is ::having a great Time, at the park time?",
           "She, unlike most women, is a big player on the park's grass.",
           "she can't be going"]

In \[18\]:

    count_vec = CountVectorizer(stop_words="english", analyzer='word',ngram_range=(1, 1), max_df=1.0, min_df=0.6, max_features=None)

    count_train = count_vec.fit(txt)
    bag_of_words = count_vec.transform(txt)

    print(count_vec.get_feature_names())
    print("\nOnly 'park' becomes the vocabulary of the document term matrix (dtm) because it appears in 2 out of 3 documents, \
    meaning 0.66% of the time.\
          \nThe rest of the words such as 'big' appear only in 1 out of 3 documents, meaning 0.33%. which is why they don't appear")

    ['park']

    Only 'park' becomes the vocabulary of the document term matrix (dtm) because it appears in 2 out of 3 documents, meaning 0.66% of the time.      
    The rest of the words such as 'big' appear only in 1 out of 3 documents, meaning 0.33%. which is why they don't appear

In \[19\]:

    count_vec = CountVectorizer(stop_words="english", analyzer='word',ngram_range=(1, 1), max_df=0.50, min_df=1, max_features=None)

    count_train = count_vec.fit(txt)
    bag_of_words = count_vec.transform(txt)

    print(count_vec.get_feature_names())
    print("\nOnly 'park' is ignored because it appears in 2 out of 3 documents, meaning 0.66% of the time.")

    ['big', 'going', 'grass', 'great', 'having', 'player', 'time', 'unlike', 'women']

    Only 'park' is ignored because it appears in 2 out of 3 documents, meaning 0.66% of the time.

In \[21\]:

    import math
    from textblob import TextBlob as tb

    def tf(word, blob):
        return blob.words.count(word) / len(blob.words)

    def n_containing(word, bloblist):
        return sum(1 for blob in bloblist if word in blob.words)

    def idf(word, bloblist):
        return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

    def tfidf(word, blob, bloblist):
        return tf(word, blob) * idf(word, bloblist)

In \[22\]:

    document1 = tb("""YouTube is an American online video-sharing platform headquartered in San Bruno, California.
    The service, created in February 2005 by three former PayPal employees—Chad Hurley, Steve Chen, and Jawed Karim—was bought
    by Google in November 2006 for US$1.65 billion and now operates as one of the company's subsidiaries. 
    YouTube is the second most-visited website after Google Search, according to Alexa Internet rankings.[7].""")

    document2 = tb("""YouTube is an American online video-sharing platform headquartered in San Bruno,
    California. The service, created in February 2005 by three former PayPal employees—Chad Hurley, 
    Steve Chen, and Jawed Karim—was bought by Google in November 2006 for US$1.65 billion and now 
    operates as one of the company's subsidiaries. YouTube is the second most-visited website after 
    Google Search, according to Alexa Internet rankings.[7]""")

    document3 = tb("""All YouTube users can upload videos up to 15 minutes each in duration. 
    Users can verify their account, normally through a mobile phone, to gain the ability to 
    upload videos up to 12 hours in length, as well as produce live streams.[100][101] 
    When YouTube was launched in 2005, it was possible to upload longer videos, but a ten-minute limit
    was introduced in March 2006 after YouTube found that the majority of videos exceeding this length were
    unauthorized uploads of television shows and films.[102] The 10-minute limit was increased to 15 minutes in July 2010.[103
    ] In the past, it was possible to upload videos longer than 12 hours. Videos can be at most 128 GB in size.[100] 
    Video captions are made using speech recognition technology when uploaded. Such captioning is usually not perfectly 
    accurate, so YouTube provides several options for manually entering the captions for greater accuracy.[104] YouTube 
    formerly offered a 'Community Captions' feature, where viewers could write and submit captions for public display upon
    approval by the video uploader, but this was deprecated on September 28, 2020. (Captions that have already been added
    using the feature will be retained..""")

    bloblist = [document1, document2, document3]
    for i, blob in enumerate(bloblist):
        print("Top words in document {}".format(i + 1))
        scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
        sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        for word, score in sorted_words[:3]:
            print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

    Top words in document 1
        Word: an, TF-IDF: 0.0
        Word: American, TF-IDF: 0.0
        Word: online, TF-IDF: 0.0
    Top words in document 2
        Word: an, TF-IDF: 0.0
        Word: American, TF-IDF: 0.0
        Word: online, TF-IDF: 0.0
    Top words in document 3
        Word: In, TF-IDF: 0.01456
        Word: videos, TF-IDF: 0.01248
        Word: was, TF-IDF: 0.01248

In \[ \]: